# Lab Assignment 4: Using APIs in Python
## DS 6001: Practice and Application of Data Science  
name: Clair McLafferty  
computing ID: cm2rh

### Instructions
Please answer the following questions as completely as possible using text, code, and the results of code as needed. Format your answers in a Jupyter notebook. To receive full credit, make sure you address every part of the problem, and make sure your document is formatted in a clean and professional way.

In this lab, you will work with the public API provided by [genius.com](https://genius.com/), a website that calls itself "the world’s biggest collection of song lyrics and musical knowledge." You will need to read the API documentation carefully, acquire an access key, and use it without sharing it to pull data from this API into Python. You will also practice using a library made specifically to wrap around `requests` to make calling from the Genius API easier.

## Problem 0
Import the following libraries:

In [2]:
import numpy as np
import pandas as pd
import requests
import json
import os
import dotenv
import sys
sys.tracebacklimit = 0 # turn off the error tracebacks

## Problem 1
The Genius API documentation is here: https://docs.genius.com/#/getting-started-h1. Read through the documentation carefully. Although the Genius API is free and public, it still requires users an access key to use the API. In this case, Genius provides users with three codes: a client ID, a client ID secret, and a client access token. Use the documentation to find a way to obtain these codes for yourself. Write a paragraph that describes all of the steps you needed to take (but DO NOT list your access codes in this paragraph).

Some hints and cautions: 

1. Before you can use the API, you will need a regular, free user account with Genius. Sign-up here: https://genius.com/signup_or_login

2. Genius's API is built to support third-party app development, not data scientists. The language is entirely geared toward app development. Under "Authentication" there are instructions to third-party developers for guiding their own users in getting access keys. That's not relevant to getting access for yourself. It's not hard to get an access key, but the guidance here is not very clear. Be patient and read everything in the Authentication section carefully.

3. When you arrive at the page that allows you to register for API access keys, the language is still geared toward app development. You will be prompted to name your app and provide the URLs associated with the app. It doesn't much matter what you name your app, and I just used the Collab main page (https://collab.its.virginia.edu/portal?containerLogin=true) for the URLs.

4. When you get your codes, copy them in a text file. In problem 2 you will copy these codes over again to a `.env` file. [4 points]

> To get the codes for the API, you must first create a free account with Genius. Although reading through the "Getting Started" page will hopefully be helpful for future steps, it was only necessary to click through the link at the top of the documentation, enter a prospective app name (mine was "DS Boot Scootin' Boogie"), and then provide the URL of the website for the app and for its implementation (??). The client ID, key, and access code are then generated and are available to copy.

## Problem 2
Create a `.env` file for this project. Open it, copy your access codes into it, and save it. Then use Python code to load the environmental variables contained in the `.env` file, and create variables that contain each of the three codes. (You can print these variables to make sure it worked, but do not allow your access keys to display in your notebook). [4 points]

In [16]:
dotenv.load_dotenv()
# set variables to different codes in .env file
client_ID = os.getenv('client_id')
client_secret = os.getenv('client_secret')
access_token = os.getenv('access_token')

## Problem 3
The root for all Genius APIs is https://api.genius.com. Find the endpoint for the Search API. (You will have to click the "Authorize with Genius" button in the upper-right corner if you haven't already done so). Use the `requests` library to issue a search for Bob Dylan. Genius's API is organized in a way that every individual artist has his or her own API endpoint. Display a portion of the JSON output that displays the API endpoint path for the data on Bob Dylan. 

Hint: to authenticate, specify your access token (not your client ID or client secret) as the `access_token` parameter. You will have to dig around the JSON output to find the artist ID, but it is listed under `primary_artist` several branches down the JSON tree. [4 points]

In [134]:
website = "https://api.genius.com/search?"
access = {"q": "Bob Dylan",
 "access_token": access_token}
r = requests.get(website, params=access) # set 
r2 = json.loads(r.text)
# found nesting data path through trial and error
partial_path = r2["response"]["hits"][0]["result"]["primary_artist"]
print(partial_path )
dylan = r2["response"]["hits"][0]["result"]["primary_artist"]["api_path"]
print("\nBob Dylan is given by artist path:", dylan) #display artist number

{'api_path': '/artists/181', 'header_image_url': 'https://images.genius.com/facc753d420efc53bbe1e0b63a72d70b.960x719x1.jpg', 'id': 181, 'image_url': 'https://images.genius.com/22306423b6ad8777d1ed5b33ad8b0d0b.1000x1000x1.jpg', 'is_meme_verified': False, 'is_verified': False, 'name': 'Bob Dylan', 'url': 'https://genius.com/artists/Bob-dylan'}

Bob Dylan is given by artist path: /artists/181


## Problem 4
Add `/songs` to the end of the the endpoint path you found in problem 3 and use this path to request the 20 most popular Bob Dylan songs. Organize these data in a `pandas` data frame. [4 points]

In [111]:
new_site = "https://api.genius.com/artists/181/songs" # update URL
songs_access = {"sort": "popularity",
                "per_page": "20", 
               "access_token": access_token}  # set new parameters
bob = requests.get(new_site, params=songs_access)
# load the file as a json, normalize, and preview
bob = json.loads(bob.text)
bobs_ur_uncle = pd.json_normalize(bob["response"]["songs"][n] for n in range(20))
bobs_ur_uncle.head(5)

,annotation_count,api_path,full_title,header_image_thumbnail_url,header_image_url,id,lyrics_owner_id,lyrics_state,path,pyongs_count,...,stats.hot,stats.pageviews,primary_artist.api_path,primary_artist.header_image_url,primary_artist.id,primary_artist.image_url,primary_artist.is_meme_verified,primary_artist.is_verified,primary_artist.name,primary_artist.url
0,16,/songs/96286,We Are the World by USA For Africa,https://images.genius.com/cd231dd0e0159b8c8b25...,https://images.genius.com/cd231dd0e0159b8c8b25...,96286,4733728,complete,/Usa-for-africa-we-are-the-world-lyrics,29,...,False,417534,/artists/370890,https://images.genius.com/3fb2d9f68c911b547339...,370890,https://images.genius.com/3fb2d9f68c911b547339...,False,False,USA For Africa,https://genius.com/artists/Usa-for-africa
1,12,/songs/79424,Blowin' in the Wind by Bob Dylan,https://images.genius.com/afd28edb1b2d6d5d556a...,https://images.genius.com/afd28edb1b2d6d5d556a...,79424,73267,complete,/Bob-dylan-blowin-in-the-wind-lyrics,43,...,False,384430,/artists/181,https://images.genius.com/facc753d420efc53bbe1...,181,https://images.genius.com/22306423b6ad8777d1ed...,False,False,Bob Dylan,https://genius.com/artists/Bob-dylan
2,1,/songs/68146,Make You Feel My Love by Adele,https://images.genius.com/e23228f25b76446cf8de...,https://images.genius.com/e23228f25b76446cf8de...,68146,82481,complete,/Adele-make-you-feel-my-love-lyrics,46,...,False,341574,/artists/2300,https://images.genius.com/2a44416083dd36132512...,2300,https://images.genius.com/2dfd665a4ae64d9e2353...,False,False,Adele,https://genius.com/artists/Adele
3,17,/songs/62605,The Times They Are A-Changin' by Bob Dylan,https://images.genius.com/bb0d6623d70dfad2803c...,https://images.genius.com/bb0d6623d70dfad2803c...,62605,49202,complete,/Bob-dylan-the-times-they-are-a-changin-lyrics,36,...,False,306348,/artists/181,https://images.genius.com/facc753d420efc53bbe1...,181,https://images.genius.com/22306423b6ad8777d1ed...,False,False,Bob Dylan,https://genius.com/artists/Bob-dylan
4,6,/songs/103982,Knockin’ on Heaven’s Door by Guns N’ Roses (Ft...,https://images.genius.com/cb2614550098bf753ab3...,https://images.genius.com/cb2614550098bf753ab3...,103982,168784,complete,/Guns-n-roses-knockin-on-heavens-door-lyrics,26,...,False,295788,/artists/637,https://images.genius.com/151b716749a32d2e0d45...,637,https://images.genius.com/6a38c80ced1ad68fa66f...,False,False,Guns N’ Roses,https://genius.com/artists/Guns-n-roses


## Problem 5
Install and import the `lyricsgenius` library in Python, which is a wrapper around `requests` that works specifically with the Genius API. . Follow the guide on the GitHub repository for this library (https://github.com/johnwmillr/LyricsGenius) for instructions on using the library. Use the `lyricsgenius` library to download and display the lyrics to "Tangled Up in Blue" by Bob Dylan. [4 points]

In [110]:
import lyricsgenius

genius = lyricsgenius.Genius(access_token) # initialize
song = genius.search_song("Tangled Up in Blue", artist.name) # search song
print(song.lyrics) # print lyrics

Searching for "Tangled Up in Blue" by Bob Dylan...
Done.
[Verse 1]
Early one morning the sun was shining
I was laying in bed
Wondering if she'd changed at all
If her hair was still red
Her folks they said our lives together
Sure was going to be rough
They never did like Mama's homemade dress
Papa's bankbook wasn't big enough
And I was standing on the side of the road
Rain falling on my shoes
Heading out for the East Coast
Lord knows I've paid some dues
Getting through
Tangled up in blue

[Verse 2]
She was married when we first met
Soon to be divorced
I helped her out of a jam, I guess
But I used a little too much force
We drove that car as far as we could
Abandoned it out West
Split up on a dark sad night
Both agreeing it was best
She turned around to look at me
As I was walking away
I heard her say over my shoulder
"We'll meet again someday
On the avenue"
Tangled up in blue

[Verse 3]
I had a job in the great north woods
Working as a cook for a spell
But I never did like it all that m

$$ 
L = - \frac{1}{M} \sum_{i=1}^m [y_i \log \, \hat{p}_i + (1 - y_i) \log \, (1 - \hat{p}_i)]
$$